In [ ]:
from numba import cffi_support, types, jit
from cffi import FFI
%matplotlib inline
from matplotlib import pyplot as plt

## Create an out-of-line module wrapping Intel's Vector Math Library

CFFI can't handle preprocessor directives, a few of which are used in the MKL headers.

So we use a trick: let's include the header and run the preprocessor over it, to get out something with no preprocessor directives in it that CFFI can read:

In [ ]:
%%writefile vml_cffi.h
#include "mkl_vml.h"

In [ ]:
!gcc -E vml_cffi.h -o vml_functions.h -I/home/pydata/anaconda3/envs/pydata/include

Now let's read in the source (ignoring any lines that begin with a #):

In [ ]:
lines = []
with open('vml_functions.h') as f:
    for line in f:
        if line[0] != '#':
            lines.append(line)
source = "".join(lines)
#print(source)

Now we have the source available, we can make a CFFI module with it:

In [ ]:
ffi = FFI()
ffi.set_source('vmlfuncs', source, libraries=['mkl_rt'])
ffi.cdef(source)
ffi.compile()

Now we can import the generated module:

In [ ]:
import vmlfuncs
# The lib member contains all the functions from the header:
vml = vmlfuncs.lib
# The ffi member contains useful functions for working with the module
ffi = vmlfuncs.ffi

Instantly, we can use these functions from Python:

In [ ]:
import numpy as np
points = np.linspace(-2 * np.pi, 2 * np.pi, 1000000)
result = np.zeros_like(points)

vml.vdSin(len(points), ffi.from_buffer(points), ffi.from_buffer(result))

plt.plot(points, result)

A performance difference is observed when using VML:

In [ ]:
points = np.linspace(-2 * np.pi, 2 * np.pi, 100000000)
result = np.zeros_like(points)

In [ ]:
%timeit vml.vdSin(len(points), ffi.from_buffer(points), ffi.from_buffer(result))

In [ ]:
%timeit np.sin(points)

## Using the module with Numba

Let's define a jitted function that uses a function from our CFFI-wrapped Vector Maths Library:

In [ ]:
vdSin = vmlfuncs.lib.vdSin

@jit(nopython=True)
def vml_sin_from_numba(x):
    out = np.zeros_like(x)
    vdSin(len(x), ffi.from_buffer(x), ffi.from_buffer(out))
    return out

And try to call it:

In [ ]:
vml_sin_from_numba(points)

The `TypingError` is because Numba needs to know a little about the type information of an out-of-line FFI module. To give it the required information, we can use Numba's `register_module` function:

In [ ]:
cffi_support.register_module(vmlfuncs)

Now let's try again:

In [ ]:
vml_sin_from_numba(points)

## Registering types

Some libraries will accept arrays of structures - for example, MKL represents complex numbers using structs like:

```
struct _MKL_Complex16 {
    double real;
    double imag;
} MKL_Complex16;
```

This coincides with the format of complex arrays in Numpy, which are stored in memory as pairs of real and imaginary parts. Logically we can pass these to VML but Numba needs to understand how to map the Numpy type to the C type.

To give it this information, we can use the `register_type` function that it provides:

In [ ]:
cffi_support.register_type(ffi.typeof('struct _MKL_Complex16'), types.complex128)

Now we should be able to use a complex-argument function from a Numba-jitted function:

In [ ]:
vzSin = vmlfuncs.lib.vzSin

@jit(nopython=True)
def vml_complex_sin_from_numba(x):
    out = np.zeros_like(x)
    vzSin(len(x), ffi.from_buffer(x), ffi.from_buffer(out))
    return out

In [ ]:
c_points = np.linspace(-2 * np.pi, 2 * np.pi, 1000000) + 1j
result = vml_complex_sin_from_numba(c_points)

In [ ]:
# Plot a subset of points (1000000 is a lot to plot)
plt.scatter(result.real[::10000], result.imag[::10000])